<a href="https://colab.research.google.com/github/dunomi/zw/blob/master/z9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')
USA_state_names= pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/states.csv')
USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

Dane pogodowe zostały pozyskane ze strony https://www.timeanddate.com/ a dane dot. rozprzestrzeniania się wirusa zostały pozyskane z bazy danych umieszczonej na stronie https://github.com/, która obecnie nie jest dostępna.

Końcowa ramka danych *USAdata* prezentuje dane dla każdego stanu USA nt. Przyrostów zachorować przez piewrszy tydzień od pierwszego wystąpienia w danym stanie zachorowania, wilgotność powietrza oraz temperaturę z danego tygodnia. Dodatkowo tabela zawiera oblcizoną korelację między Temperaturą/Wilgotnością a Przyrostami zachorowań oraz kilka innych zmiennych pomocniczych służących do tworzenia wykresów.

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)

USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))
USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]

USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100
  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
USAdata=pd.merge(USAcases,USAweather)

USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

#USAcumdata.loc[USAcumdata.Przyrosty > 50, 'Przyrosty'] = max(USAcumdata.loc[USAcumdata.Przyrosty < 50].Przyrosty)

for i in range(len(USAdata)):
  USAdata.loc[i, 'Temperatura_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Temperatura[i][0:6].astype(float))[0,1]
  USAdata.loc[i, 'Wilgotnosc_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Wilgotnosc[i][0:6].astype(float))[0,1]

  USAdata.loc[i, 'Temperatura_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Temperatura'][i][0:6].tolist())).fit().params[1]
  USAdata.loc[i, 'Wilgotnosc_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Wilgotnosc'][i][0:6].tolist())).fit().params[1]

  USAdata.loc[i, 'Przyrosty_wykres_osY']=(max(USAdata.Przyrosty[i])-min(USAdata.Przyrosty[i]))/2
  USAdata.loc[i, 'Temperatura_wykres_osX']=(max(USAdata.Temperatura[i])-min(USAdata.Temperatura[i]))/2
  USAdata.loc[i, 'Wilgotnosc_wykres_osX']=(max(USAdata.Wilgotnosc[i])-min(USAdata.Wilgotnosc[i]))/2

**Wykres 1**

Poniżej prezentują się zbiory wykresów na których na osi X znajduje się odpowiednio Temperatura (dla górnego wykresu) i Wilgotnośc (dla dolnego wykresu), a na osi Y Przyrost zachorowań. Każdy wykres dla innego stanu. Prosta linia to regresja liniowa wyznaczona dla Przyrostów i Temperatury/Wilgotności.

In [0]:
def plot_maker(Data,Temperatura_lub_Wilgotnosc):
  
  RangeY_max=max(Data['Przyrosty_wykres_osY'].tolist())
  RangeX_max=max(Data[Temperatura_lub_Wilgotnosc+'_wykres_osX'].tolist())

  fig = make_subplots(rows=10, cols=5, shared_xaxes=False,  x_title=Temperatura_lub_Wilgotnosc, y_title='Przyrosty',)
  for i in range(0,len(Data)):
    x=Data.loc[i]

    RangeY=[min(x['Przyrosty'])+x['Przyrosty_wykres_osY']-RangeY_max,
            min(x['Przyrosty'])+x['Przyrosty_wykres_osY']+RangeY_max]  
    RangeX=[min(x[Temperatura_lub_Wilgotnosc])+x[Temperatura_lub_Wilgotnosc+'_wykres_osX']-RangeX_max,
            min(x[Temperatura_lub_Wilgotnosc])+x[Temperatura_lub_Wilgotnosc+'_wykres_osX']+RangeX_max]

    figX = px.scatter(x=x[Temperatura_lub_Wilgotnosc].tolist()[0:6],
                      y=x['Przyrosty'].tolist(),
                      trendline="ols")


    tracePoints = figX['data'][0]
    traceTradeline = figX['data'][1]

    fig.add_trace(tracePoints, row=math.floor(i/5)+1, col=i%5+1)
    fig.add_trace(traceTradeline, row=math.floor(i/5)+1, col=i%5+1)

    fig.update_yaxes(range=[0, 5], row=math.floor(i/5)+1, col=i%5+1)
    fig.update_xaxes(range=RangeX, row=math.floor(i/5)+1, col=i%5+1)


  fig['layout'].update(height=1500, width=1000, showlegend=False, title=Temperatura_lub_Wilgotnosc)

  return fig.show()

#Wykresy posortowane po współczynniku regresji liniowej
plot_maker(USAdata.sort_values(by='Temperatura_cor_wsp_ols').reset_index().drop(columns='index'),'Temperatura')
plot_maker(USAdata.sort_values(by='Wilgotnosc_cor_wsp_ols').reset_index().drop(columns='index'),'Wilgotnosc')

**Wykres 2**

Poniższe wykresy prezentują odpowiednio Temperaturę (dla górnego wykresu) i Wilgotność (dla dolnego wykresu), a na osi Y Przyrost zachorowań dla każdego Stanu.

Trzeci wykres łączy dwa poprzedzające. 

In [0]:
fig1 = px.scatter(x=USAcumdata['Temperatura'][USAcumdata['Przyrosty']<100].tolist(), 
                 y=USAcumdata['Przyrosty'][USAcumdata['Przyrosty']<100].tolist(),
                 trendline="ols",
                 labels={'x':'Temperatura', 'y':'Przyrosty'})
fig1.show()

fig2 = px.scatter(x=USAcumdata['Wilgotnosc'][USAcumdata['Przyrosty']<100].tolist(), 
                 y=USAcumdata['Przyrosty'][USAcumdata['Przyrosty']<100].tolist(),
                 trendline="ols",
                 labels={'x':'Wilgotnosc', 'y':'Przyrosty'})
fig2.show()

##
fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig1['data'][1], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig2['data'][1], row=1, col=2)

fig.update_layout( yaxis_title="Regresja liniowa", bargroupgap=0.1)

fig.show()


**Wykres 3**

Wykresy mapowe prezentują tygodniowe Przyrosty zachorować, średnią Temperaturę oraz średnią Wilgotność dla każdego Stanu.

In [0]:
USAcumdata_map=USAcumdata.reset_index()[['Stan','Przyrosty','Temperatura','Wilgotnosc']]

new_column = pd.Series([30,30,30,30], name='Przyrosty', index=USAcumdata_map[USAcumdata_map['Przyrosty']>30].index)
USAcumdata_map.update(new_column)

USA_us_map=USAcumdata_map.merge(USA_state_names, left_on='Stan', right_on='State')

figSubPlot = make_subplots(rows=2, cols=4, shared_xaxes=False)

fig1 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Przyrosty'], scope="usa",title='Przyrosty')
fig2 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Temperatura'], scope="usa",title='Temperatura')
fig3 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Wilgotnosc'], scope="usa",title='Wilgotnosc')

fig1.show()
fig2.show()
fig3.show()

**Wykres 4**

Poniższy wykres prezentuje zobrazowanie trzech zmiennych na wykresie dwuwymiarowym dla tygodniowego Przyrostu zachorowań (wielkość kropek), średniej Temperatury (oś X) oraz średniej Wilgotności (oś Y) dla każdego stanu.

In [0]:
fig = px.scatter(x=USAcumdata['Temperatura'].tolist(), 
                 y=USAcumdata['Wilgotnosc'].tolist(),
                 size=USAcumdata['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.update_yaxes(range=[0,1])
fig.show()


**Wykres 5**

Poniższy wykres słupkowy obrazuje ilość wyników obliczonych korelacji dla Przyrostów zachorowań i odpowiednio Temperatury (dla lewego wykresu) i Wilgotności (dla prawego wykresu), które to wyniki wpadły w odpowiedni przedział.
Zaobserwować

In [0]:
lista_temperatura=[]
lista_wilgotnosc=[]
for i in range(len(USAdata)):
  t=USAdata.loc[i]
  lista_temperatura.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Temperatura[0:6]}).astype(float).corr()['x'].loc['y'])
  lista_wilgotnosc.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Wilgotnosc[0:6]}).astype(float).corr()['x'].loc['y'])

lista_temperatura.sort()
lista_wilgotnosc.sort()

fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

fig_temperatura = px.histogram(x=lista_temperatura,nbins=30)
fig_wilgotnosc = px.histogram(x=lista_wilgotnosc,nbins=30)

fig.add_trace(fig_temperatura['data'][0], row=1, col=1)
fig.add_trace(fig_wilgotnosc['data'][0], row=1, col=2)

fig.update_layout( yaxis_title="Ilość korelacji", bargroupgap=0.1)

fig.show()

In [31]:
USAdata

,Stan,Data,Przyrosty,Temperatura,Wilgotnosc,Temperatura_cor,Wilgotnosc_cor,Temperatura_cor_wsp_ols,Wilgotnosc_cor_wsp_ols,Przyrosty_wykres_osY,Temperatura_wykres_osX,Wilgotnosc_wykres_osX
0,Washington,3/10/20,"[1.37, 1.21, 1.29, 1.01, 1.12, 1.41]","[12, 12, 11, 3, 6, 8, 14]","[0.45, 0.45, 0.46, 0.91, 0.55, 0.38, 0.33]",0.700258,-0.863746,0.029109,-0.685112,0.200,5.5,0.290
1,New York,3/10/20,"[1.27, 1.49, 1.28, 1.25, 1.39, 1.32]","[17, 8, 11, 13, 7, 8, 8]","[0.56, 0.36, 0.4, 0.63, 0.44, 0.27, 0.21]",-0.687850,-0.522874,-0.016409,-0.361450,0.120,5.0,0.210
2,California,3/10/20,"[1.23, 1.25, 1.28, 1.21, 1.25, 1.31]","[19, 22, 23, 18, 14, 11, 13]","[0.45, 0.48, 0.38, 0.58, 0.71, 0.74, 0.63]",-0.346023,0.265789,-0.002668,0.065013,0.050,6.0,0.180
3,Massachusetts,3/10/20,"[1.03, 1.14, 1.14, 1.12, 1.19, 1.2]","[19, 11, 6, 16, 12, 5, 7]","[0.62, 0.53, 0.77, 0.84, 0.32, 0.44, 0.42]",-0.768976,-0.513798,-0.008562,-0.158507,0.085,7.0,0.260
4,Georgia,3/10/20,"[1.35, 1.35, 1.35, 1.57, 1.5, 1.22]","[18, 21, 24, 22, 27, 21, 14]","[0.76, 0.71, 0.69, 0.71, 0.49, 0.78, 0.78]",0.438933,-0.575062,0.017936,-0.693309,0.175,6.5,0.145
5,Colorado,3/10/20,"[2.27, 1.32, 1.09, 2.06, 1.3, 1.22]","[16, 20, 8, 4, 10, 15, 16]","[0.27, 0.19, 0.55, 0.71, 0.56, 0.43, 0.35]",-0.116205,-0.008396,-0.009740,-0.021219,0.590,8.0,0.260
6,Florida,3/10/20,"[1.87, 1.25, 1.43, 1.52, 1.51, 1.35]","[24, 25, 28, 27, 29, 29, 29]","[0.55, 0.57, 0.54, 0.56, 0.43, 0.4, 0.5]",-0.430073,0.182373,-0.043636,0.527385,0.310,2.5,0.085
7,New Jersey,3/10/20,"[1.53, 1.26, 1.0, 2.38, 1.42, 1.82]","[18, 13, 11, 22, 13, 12, 7]","[0.6, 0.36, 0.72, 0.57, 0.27, 0.33, 0.55]",0.777964,-0.086765,0.088055,-0.233302,0.690,7.5,0.225
8,Oregon,3/10/20,"[1.27, 1.26, 1.25, 1.07, 1.12, 1.08]","[17, 14, 12, 4, 7, 10, 16]","[0.44, 0.6, 0.49, 0.86, 0.79, 0.43, 0.54]",0.871680,-0.514925,0.017515,-0.264559,0.100,6.5,0.215
9,Texas,3/10/20,"[1.62, 1.29, 1.59, 1.33, 1.26, 1.18]","[27, 29, 30, 28, 29, 17, 23]","[0.61, 0.52, 0.49, 0.55, 0.56, 0.87, 0.74]",0.487670,-0.476894,0.018381,-0.629707,0.220,6.5,0.190
